In [3]:
import os
import yaml

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm, trange


Bad key text.latex.unicode in file /home/nhat/miniconda3/envs/salsanext/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.4/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.frameon in file /home/nhat/miniconda3/envs/salsanext/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 423 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.4/matplotlibrc.template
or from the matplotlib source distribution

Bad key pgf.debug in file /home/nhat/miniconda3/envs/salsanext/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 444 ('pgf.debug           : False')
You probably need to get

In [15]:
HOMEDIR = os.environ['HOME']
COURSEDIR = f"{HOMEDIR}/work/uq/engg4811"
DATADIR = f"{COURSEDIR}/datasets/Rellis-3D"

In [16]:
DATA_CONFIG_FILE = "../train/tasks/semantic/config/labels/rellis.yaml"
CONFIG = yaml.safe_load(open(DATA_CONFIG_FILE, 'r'))

In [17]:
# Map labels to class ids
CID_MAP = CONFIG['learning_map']

In [18]:
# Map class ids to class names
NAME_MAP = dict()
for cid, label in CONFIG['learning_map_inv'].items():
    NAME_MAP[cid] = CONFIG['labels'][label]
    
CLASS_NAMES = [NAME_MAP[i] for i in range(len(NAME_MAP))]

In [19]:
"""
Get a vector of labels
"""
def get_labels(filepath):
    return np.fromfile(filepath, dtype=np.int32).reshape((-1))

In [20]:
"""
Convert a label vector to a class id vector
"""
def labels_to_cids(labels):
    tmp = labels.copy()
    for k, v in CID_MAP.items():
        labels[tmp == k] = v
    return labels

In [49]:
def get_split_frequency():
#     assert(split_name in ["pt_train", "pt_test", "pt_val"])
    label_filenames = []

#     with open(f'{DATADIR}/{split_name}.lst', 'r') as f:
    with open(f'{split_name}.lst', 'r') as f:
        for line in f:
            label_filename = line.strip().split(' ')[1]
            label_filenames.append(label_filename)

    n = len(CLASS_NAMES)
    count = np.zeros(n, dtype=np.int32)

    for fname in tqdm(label_filenames):
        glabels = get_labels(f'{DATADIR}/{fname}')
        g = labels_to_cids(glabels)
        bins = np.bincount(g)
        bins = np.pad(bins, (0, n - len(bins)), 'constant')
        count = count + bins

    return count

In [25]:
fval = get_split_frequency('pt_val')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2034/2034 [00:03<00:00, 555.11it/s]


In [26]:
for i, c in enumerate(CLASS_NAMES):
    print(f'class: {c}    => {fval[i]}')

class: void    => 139221198
class: grass    => 13829166
class: tree    => 8835696
class: pole    => 0
class: water    => 0
class: vehicle    => 0
class: log    => 0
class: person    => 22374
class: fence    => 0
class: bush    => 97534368
class: concrete    => 2808954
class: barrier    => 518670
class: puddle    => 26442
class: mud    => 3803580
class: rubble    => 0


In [27]:
ftrain = get_split_frequency('pt_train')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9486/9486 [00:18<00:00, 517.71it/s]


In [28]:
for i, c in enumerate(CLASS_NAMES):
    print(f'class: {c}    => {ftrain[i]}')

class: void    => 629282268
class: grass    => 459625158
class: tree    => 42876720
class: pole    => 0
class: water    => 0
class: vehicle    => 768366
class: log    => 0
class: person    => 3395988
class: fence    => 0
class: bush    => 93731166
class: concrete    => 123318
class: barrier    => 0
class: puddle    => 2191266
class: mud    => 11354742
class: rubble    => 0


In [30]:
ftest = get_split_frequency('pt_test')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2036/2036 [00:04<00:00, 466.47it/s]


In [31]:
for i, c in enumerate(CLASS_NAMES):
    print(f'class: {c}    => {ftest[i]}')

class: void    => 95042516
class: grass    => 65318952
class: tree    => 53192536
class: pole    => 0
class: water    => 0
class: vehicle    => 0
class: log    => 0
class: person    => 1050576
class: fence    => 0
class: bush    => 48477160
class: concrete    => 0
class: barrier    => 0
class: puddle    => 12216
class: mud    => 3764564
class: rubble    => 4072


In [15]:
from os import listdir
from os.path import isfile, join

def get_sequence_frequency(seq):
    assert(seq in ['00000', '00001', '00002', '00003', '00004'])
    seq_dir = f'{DATADIR}/{seq}/os1_cloud_node_semantickitti_label_id/'
    files = [f for f in listdir(seq_dir) if isfile(join(seq_dir, f))]

    n = len(CLASS_NAMES)
    count = np.zeros(n, dtype=np.int32)

    for filename in tqdm(files):
        glabels = get_labels(f'{seq_dir}/{filename}')
        g = labels_to_cids(glabels)
        bins = np.bincount(g)
        bins = np.pad(bins, (0, n - len(bins)), 'constant')
        count = count + bins
    
    
    fig = plt.figure()
    ax = fig.add_axes([0, 0, 2, 2])
    ax.bar(CLASS_NAMES, count)
    ax.set_xlabel('Semantic class')
    ax.set_ylabel('Fequency (number of points)')
    ax.set_title(f'Frequency of all semantic classes - Sequence {seq}')
    for index, value in enumerate(count): 
        ax.text(index, value, str(count[index]), ha='center')
    plt.show()

    return count

In [ ]:
NUM_SEQS = 5
f = [None for _ in range(NUM_SEQS)]

for k in range(NUM_SEQS):
    f[k] = get_freq_seq(f'0000{k}')
    print(f"Sequence 0000{k}")
    for i, c in enumerate(CLASS_NAMES):
        print(f'class: {c}    => {f[k][i]}')

In [32]:
import random
from os import listdir
from os.path import isfile, join

def gen_split(seed=124, split_ratios=[0.7, 0.85]):
    random.seed(seed)
    train_split_lines = []
    val_split_lines = []
    test_split_lines = []
    bin_dir = 'os1_cloud_node_kitti_bin'
    label_dir = 'os1_cloud_node_semantickitti_label_id'
    NUM_SEQS = 5
    for k in tqdm(range(NUM_SEQS)):
        seq = f'0000{k}'
        print(f'Sequence: {seq}', end=' - ')
        
        seq_dir = f'{seq}/os1_cloud_node_semantickitti_label_id'
        abs_seq_dir = f'{DATADIR}/{seq_dir}'
        
        scans = [f[:-6] for f in listdir(abs_seq_dir) if isfile(join(abs_seq_dir, f))]
        random.shuffle(scans)
        split_idx = [int(len(scans) * r) for r in split_ratios]
        print(f'split indices: ({split_idx[0]}, {split_idx[1]})')
        
        train_split = scans[:split_idx[0]]
        val_split = scans[split_idx[0]:split_idx[1]]
        test_split = scans[split_idx[1]:]
        
        train_split_lines += [' '.join([f'{seq}/{bin_dir}/{scan}.bin', f'{seq}/{label_dir}/{scan}.label']) for scan in train_split]
        val_split_lines += [' '.join([f'{seq}/{bin_dir}/{scan}.bin', f'{seq}/{label_dir}/{scan}.label']) for scan in val_split]
        test_split_lines += [' '.join([f'{seq}/{bin_dir}/{scan}.bin', f'{seq}/{label_dir}/{scan}.label']) for scan in test_split]
    
    random.shuffle(train_split_lines)
    random.shuffle(val_split_lines)
    random.shuffle(test_split_lines)
    
    with open('pt_train.lst', 'w') as f:
        for line in train_split_lines:
            f.write(line + '\n')
    
    with open('pt_val.lst', 'w') as f:
        for line in val_split_lines:
            f.write(line + '\n')
            
    with open('pt_test.lst', 'w') as f:
        for line in test_split_lines:
            f.write(line + '\n')

In [33]:
gen_split()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 45.44it/s]

Sequence: 00000 - split indices: (1992, 2419)
Sequence: 00001 - split indices: (1623, 1971)
Sequence: 00002 - split indices: (2902, 3524)
Sequence: 00003 - split indices: (1528, 1856)
Sequence: 00004 - split indices: (1441, 1750)


In [38]:
ftrain = get_freq_split('pt_train')
for i, c in enumerate(CLASS_NAMES):
    print(f'class: {c} '.ljust(20) + f'=> {ftrain[i]}')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9486/9486 [00:19<00:00, 483.99it/s]

class: void         => 629282268
class: grass        => 459625158
class: tree         => 42876720
class: pole         => 0
class: water        => 0
class: vehicle      => 768366
class: log          => 0
class: person       => 3395988
class: fence        => 0
class: bush         => 93731166
class: concrete     => 123318
class: barrier      => 0
class: puddle       => 2191266
class: mud          => 11354742
class: rubble       => 0


In [39]:
fval = get_freq_split('pt_val')
for i, c in enumerate(CLASS_NAMES):
    print(f'class: {c} '.ljust(20) + f'=> {fval[i]}')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2034/2034 [00:03<00:00, 530.54it/s]

class: void         => 139221198
class: grass        => 13829166
class: tree         => 8835696
class: pole         => 0
class: water        => 0
class: vehicle      => 0
class: log          => 0
class: person       => 22374
class: fence        => 0
class: bush         => 97534368
class: concrete     => 2808954
class: barrier      => 518670
class: puddle       => 26442
class: mud          => 3803580
class: rubble       => 0


In [40]:
ftest = get_freq_split('pt_test')
for i, c in enumerate(CLASS_NAMES):
    print(f'class: {c} '.ljust(20) + f'=> {ftest[i]}')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2036/2036 [00:04<00:00, 444.37it/s]

class: void         => 95042516
class: grass        => 65318952
class: tree         => 53192536
class: pole         => 0
class: water        => 0
class: vehicle      => 0
class: log          => 0
class: person       => 1050576
class: fence        => 0
class: bush         => 48477160
class: concrete     => 0
class: barrier      => 0
class: puddle       => 12216
class: mud          => 3764564
class: rubble       => 4072


In [11]:
np.maximum(1, 2)

2

In [43]:
a = [i for i in range(20)]
split_ratios=[0.7, 0.85]
split_idx = [int(len(a) * r) for r in split_ratios]
print(f'split indices: ({split_idx[0]}, {split_idx[1]})')

split indices: (14, 17)


In [44]:
train_split = a[:split_idx[0]]
val_split = a[split_idx[0]:split_idx[1]]
test_split = a[split_idx[1]:]

In [45]:
train_split

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [46]:
val_split

[14, 15, 16]

In [47]:
test_split

[17, 18, 19]